In [12]:
import argparse, time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
from dgl.data import register_data_args, load_data
from dgl.data import BitcoinOTC
import datetime
from dgl.nn.pytorch import GraphConv
import time
from sklearn.metrics import f1_score
import os
import json
from collections import defaultdict, Counter
from tqdm import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import networkx as nx
from dgl import DGLGraph
from dgl.nn.pytorch.conv import SAGEConv

## Hyperparameters

In [ ]:
node_dim = 256
hid_dim = 256
mlp_dim = 256
n_layers = 2
dropout = 0.2
learning_rate = 0.01
wt_decay = 5e-4
stpsize = 15
n_epochs = 10
out_path = '../btc_static/'
self_loop = True
data_path = '../soc-sign-bitcoinotc.csv'

## Data loading

In [11]:
def load_hate(features, edges, num_features):
    num_nodes = 100386
    num_feats = num_features
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes, 1), dtype=np.int64)
    node_map = {}
    label_map = {}

    with open(features) as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            feat_data[i, :] = list(map(float, info[1:-1]))
            node_map[info[0]] = i
            if not info[-1] in label_map:
                label_map[info[-1]] = len(label_map)
            labels[i] = label_map[info[-1]]

    adj_lists = defaultdict(set)
    with open(edges) as fp:
        for i, line in enumerate(fp):
            info = line.strip().split()
            paper1 = node_map[info[0]]
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)

    print(label_map)
    return feat_data, labels, adj_lists

## Model

In [3]:
class GraphSAGE(nn.Module):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.g = g

        # input layer
        self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type, feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(SAGEConv(n_hidden, n_hidden, aggregator_type, feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(SAGEConv(n_hidden, n_classes, aggregator_type, feat_drop=dropout, activation=None)) # activation None

    def forward(self, features):
        h = features
        for layer in self.layers:
            h = layer(self.g, h)
        return h

## Training loop

In [8]:
def predict_logits(model, device, features, mask=None):
    model.eval()
    with torch.no_grad():
        features = features.to(device)
        logits = model(features)

        if mask is not None:
            logits = logits[mask]
    return logits

def evaluate(logits, labels, mask=None):
    if mask is not None:
        logits = logits[mask]
        labels = labels[mask]
    
    sigLayer = nn.Sigmoid()
    predictions_scores = sigLayer(logits).cpu().detach().numpy()
    roc_auc = metrics.roc_auc_score(labels, predictions_scores)
    
    _, indices = torch.max(logits, dim=1)
    correct = torch.sum(indices == labels)
    return (roc_auc, correct.item() * 1.0 / len(labels))

In [9]:
def evaluate_loss(model, criterion, device, val_mask, graph):
    model.eval()
    #validation phase
    with torch.set_grad_enabled(False):
        feat = graph.ndata['feat'].to(device)
        outputs = model(feat, graph)
        labels = graph.ndata['labels'].to(device)
        loss = criterion(outputs[val_mask], labels[val_mask])

    return loss.item()

In [10]:
#Code for supervised training
def train_model(model, criterion, optimizer, scheduler, device, checkpoint_path, hyperparams, num_epochs=25):
    metrics_dict = {}
    metrics_dict["train"] = {}
    metrics_dict["valid"] = {}
    metrics_dict["train"]["loss"] = {}
    metrics_dict["train"]["loss"]["epochwise"] = []
    metrics_dict["valid"]["loss"] = {}
    metrics_dict["valid"]["loss"]["epochwise"] = []
        
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{} \n'.format(epoch, num_epochs - 1))
        print('-' * 10)
        print('\n')
        
        #train phase
        scheduler.step()
        model.train() 
        optimizer.zero_grad()
        # forward
        # track history if only in train
        forward_start_time  = time.time()
        feats = graph.ndata['feat'].to(device)
        outputs = model(feats, graph)
        labels = graph.ndata['labels']
        labels = labels.to(device)
        loss = criterion(outputs[train_mask], labels[train_mask])
        epoch_loss = loss.item()
        loss.backward()
        optimizer.step()
        forward_time = time.time() - forward_start_time
        
        print('Train Loss: {:.4f} \n'.format(epoch_loss))
        metrics_dict["train"]["loss"]["epochwise"].append(epoch_loss)
        
        #validation phase
        val_epoch_loss = evaluate_loss(model, criterion, device, val_mask, graph)
        print('Validation Loss: {:.4f} \n'.format(val_epoch_loss))
        metrics_dict["valid"]["loss"]["epochwise"].append(val_epoch_loss)
        
        # deep copy the model
        if val_epoch_loss < best_loss:
            best_loss = val_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())

        torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'full_metrics': metrics_dict,
        'hyperparams': hyperparams
        }, '%s/net_epoch_%d.pth' % (checkpoint_path, epoch))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s \n'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f} \n'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
 # create GCN model
model = GraphSAGE(max_out_degrees, node_dim, hid_dim, mlp_dim, n_layers, F.relu, dropout)
model.to(device)
criterion = nn.MSELoss()
model_parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(model_parameters, lr=learning_rate, weight_decay = wt_decay)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=stpsize, gamma=0.1)
hyper_params = {'node_dim' : node_dim,
    'hid_dim': hid_dim,
    'mlp_dim': mlp_dim,
    'n_layers' : n_layers,
    'dropout' : dropout,
    'wt_decay' : wt_decay,
    'voc_size' : max_out_degrees}

bst_model = train_model(model, criterion, optimizer, exp_lr_scheduler, device, out_path, hyper_params, n_epochs)